# Investigating the relation between energy and information

## Work extraction from a generalised Szilard engine

This file contains the code for the main analysis in my thesis. It essentially runs a serial analysis, saving data files from serial analysis in folder called 'polya_images', located within the folder of this script.
These simulations compute the average work, the average entropy, the mass probability function and plot the relation between our parameter $\alpha$ and the average work as $q$ and $N$ change.

## Import libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
import pickle
import math

## Functions

In [2]:
def W(k,n,Q): #Work extracted
    w = k_b*T*sum([x*np.log(x*Q/n) for x in k if x > 0])
    
    return w

In [3]:
def F(P,n): #F function to compute average entropy
    k = np.arange(1, n+1)
    f = sum([x*math.log(x, 2.0)*P[x] for x in k])

    return f

In [4]:
def p(q,N,alpha): # Probability distribution of particles in a given partition

    P = [[1],[1-1/q, 1/q]] #initial conditions for N=1

    for n in range(1,N):

        p_temp = []

        pn_0 = P[n][0]*(1-alpha/q) #k=0 for n+1 step
        p_temp.extend([pn_0])

        for k in range(1,n+1): # 1 <= k <= n
            pn_ = P[n][k]*(1-alpha/q-(1-alpha)*k/n) + P[n][k-1]*(alpha/q + (1-alpha)*(k-1)/n)
            p_temp.extend([pn_])

        k = n+1
        pn_n = P[n][k-1]*(alpha/q + (1-alpha)*(k-1)/n)
        p_temp.extend([pn_n])

        P.append(p_temp)
        
    return P[N]

In [5]:
def initialise_batches():
    
    W_save = [] # batch to save mean work, averaged across trials, for each alpha value
    Werr = [] # errorbar
    K_save = [] # batch to save mean number of particles across compartments, averaged across trials, for each alpha value

    P_save = []
    W_save_th = []
    
    return W_save, Werr, K_save, P_save, W_save_th

In [6]:
def polya(N, alpha, comp_list_all): # placing particles
    
    K = np.zeros((N,q)) # initialise vector to store number of particles in each partition during Polya's Urn simulations
    comp_list = [] #useful list for random compartment # choice

    comp = random.choice(comp_list_all) #random compartment choice for 1st particle
    comp_list.append(comp)
    K[0, comp] = 1 #place one particle in the 1st chosen compartment at step 0
    
    for pc in range(1,N): #placement of N particles from n>=2
    
        if random.random() <= alpha:
            comp = random.choice(comp_list_all)

        else:
            comp = random.choice(comp_list)

        comp_list.append(comp)

        K[pc, :] = K[pc-1, :] #keep same number of particles for all compartments from previous step
        K[pc, comp] = K[pc-1, comp]+1 #add particle to chosen compartment
    
    return K[-1,:] #keep only last placement arrangement

In [7]:
def process(trials, N, q, alpha):

    K_temp = [] #batch to save number of particles at each trial
    W_temp = [] #batch to save extracted work at each trial

    for i in range(trials): #multiple trials for each alpha value
                
        K = polya(N, alpha, comp_list_all) # placing particles and storing their number in vector K

        W_temp.append(W(K,N,q)) #computing and saving work W
            
    return W_temp

In [8]:
def analytical_predictions(q, N, alpha):
    
    # Probability distribution of particles in a given compartment
    P = p(q, N, alpha) #compute
    P_save.append(P) #save
            
    # Average entropy
    H_mean = - q/N * F(P,N) + math.log(N, 2.0) #compute
            
    # Average work extracted
    W_mean = - N * k_b * T * np.log(2) * (H_mean - math.log(q, 2.0)) #compute
    
    return P, W_mean

In [9]:
def savefile(q, N, W_save_th, W_save, K_save, P_save):
    
    #Name of files - saving purposes
    name_file_ID = "_q" + str(q) + "_N" + str(N)
    folder = 'polya_images/'

    #Save With Parameters included in file name
    file = [q, N, W_save_th, W_save, K_save, P]

    with open(folder+name_file_ID+'.txt', "wb") as fp:   # Unpickling
        pickle.dump(file, fp)
    
    return folder, name_file_ID

In [10]:
def savefigure(Alpha, W_save, Werr, W_save_th, folder, name_file_ID):
    
    plt.ioff() #avoid showing figures

    plt.figure(figsize=(16,8))
    plt.errorbar(Alpha, W_save, yerr=Werr, color='k', alpha=.7, label='Simulations')
    plt.plot(Alpha, W_save_th, color='r', linewidth=3, label='Prediction')
    plt.xlabel(r'$ \alpha $')
    plt.ylabel('-$W (J)$')
    plt.legend()
    plt.savefig(folder+"W_alpha"+name_file_ID+".png") # save as png
    plt.close()

## Simulations

## Serial analysis: saving data and plots using different $q$ and $N$

In [11]:
#Physical constants
k_b = 1 #Boltzmann constant
T = 1 #Temperature

# Parameters for serial analysis
compartments = [2, 3, 5, 10, 100, 250, 500, 750, 1000] # number of walls for serial analysis
particles = [1, 2, 3, 5, 10, 100, 250, 500, 750, 1000] # total number of particles for serial analysis

# Paramteres for Polya's Urn simulations
trials = 1000 # trials for each alpha value
alpha_res = 51 # number of alpha values between 0 and 1; it determines the resolution of the plots
Alpha = np.linspace(0,1,alpha_res) # Polya Urn's parameter ranging between 0 and 1

# COMPUTATIONS

for idxq, q in enumerate(compartments): # for each number of compartments
    
    comp_list_all = (np.arange(q)).tolist() #useful list with all the compartment #
    
    for idxN, N in enumerate(particles): # for each number of particles

        # initialise batches
        W_save, Werr, K_save, P_save, W_save_th = initialise_batches()
        
        # POLYA'S URN
        for alpha in Alpha: # test different alpha values
            
            # SIMULATIONS
            W_temp = process(trials, N, q, alpha) #compute work for all trials
            W_save.append(np.mean(W_temp)) #save mean work
            Werr.append(np.std(W_temp)) #save SD
            
            # ANALYTICAL PREDICTIONS
            P, W_mean = analytical_predictions(q, N, alpha)
            P_save.append(P), W_save_th.append(W_mean)
            
        # SAVE FILE
        folder, name_file_ID = savefile(q, N, W_save_th, W_save, K_save, P_save)

        # SAVE FIGURE
        savefigure(Alpha, W_save, Werr, W_save_th, folder, name_file_ID)